In [35]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [36]:
import faiss 
import numpy as np
import pickle

In [37]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [38]:
class semanticEmbedding:
    def __init__(self, model_name='sentence-transformers/all-mpnet-base-v2'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    def get_embedding(self, sentences):
    # Tokenize sentences
        encoded_input = self.tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**encoded_input)
        # Perform pooling
        sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])

        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings.detach().numpy()

In [39]:
class FaissIdx:

    def __init__(self, model, dim=768):
        self.index = faiss.IndexFlatIP(dim) # need to load the pickle model in the final file
        # self.index = faiss
        # Maintaining the document data
        self.doc_map = dict()
        self.model = model
        self.ctr = 0

    def add_doc(self, document_text):
        self.index.add(self.model.get_embedding(document_text))
        self.doc_map[self.ctr] = document_text # store the original document text
        self.ctr += 1

    def search_doc(self, query, k=3):
        D, I = self.index.search(self.model.get_embedding(query), 5)
        return [{self.doc_map[idx]: score} for idx, score in zip(I[0], D[0]) if idx in self.doc_map]
    def save_index(self, index_filename, doc_map_filename):
        # Save Faiss index to file
        faiss.write_index(self.index, index_filename)

        # Save document map to file using pickle
        with open(doc_map_filename, 'wb') as f:
            pickle.dump(self.doc_map, f)

    def load_index(self, index_filename, doc_map_filename):
        # Load Faiss index from file
        self.index = faiss.read_index(index_filename)

        # Load document map from file using pickle
        with open(doc_map_filename, 'rb') as f:
            self.doc_map = pickle.load(f)

In [40]:
# creating an instance of the class
model = semanticEmbedding()

In [41]:
index = FaissIdx(model)

In [42]:
index.load_index('index.bin', 'doc_map.pkl')

In [43]:
import google.generativeai as genai

In [44]:
genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

In [45]:
model = genai.GenerativeModel('gemini-pro')

# given the query, run the following code sequentially

In [64]:
query = "I want to help poor"

In [65]:
user_name = 'Amar'

In [66]:
website = 'https://www.india.gov.in/my-government/schemes'

In [67]:
query_result = index.search_doc(query)

In [68]:
context = ''

In [69]:
for i in range(5):
    for key, value in query_result[i].items():
        if(value>0.23):
            context += key

In [70]:
context

'Dr. B.R. Ambedkar Awas Navinikarn Yojna, Haryana = The main purpose of this scheme is to provide grant for the repair of house to all families living under below poverty line, whose house needs repair.Schemes for welfare of backward classes by Ministry of Social Justice and Empowerment = Information about schemes and programmes for the welfare of backward classes by Ministry of Social Justice and Empowerment. Information about pre-matric and post-matric scholarships, centrally sponsored schemes of construction of hostels, assistance to voluntary organisations working for welfare of OBCs, etc are available. Information on National Backward Classes Finance and Development Corporation (NBCFDC), state-wise and scheme-wise expenditure under the pre-Matric and post-Matric and hostels for OBCs and contact details of concerned officers for queries can also be obtained.General Grant-in-Aid Scheme in the field of Women and Child Development = The role of voluntary organizations and their partic

In [71]:
content = ""

In [72]:
if context == '':
    print('Is there anything else I can help you with?')
else:
    formatted_response = f''' Give response in this format, dont change the format or the links given-: 
Hello {user_name},

Thank you for reaching out! 🌟 Based on the information you provided, here are some government schemes that might be relevant to your situation:

1. Scheme Name 1:
   - Information like eligibility, benefit.

2. Scheme Name 2:
   - Information like eligibility, benefit.

3. Scheme Name 3:
   - Information like eligibility, benefit.

4. Scheme Name 4:
   - Information like eligibility, benefit.

5. Scheme Name 5:
   - Information like eligibility, benefit.


For more detailed information or to apply, you can visit the official [Government Department/Agency] website: {website}.


If you have any more questions or need further assistance, feel free to ask!

Best regards,
GovSevak.

*This message is generated by GENAI, GENAI can make mistakes. Consider checking important information.*

QUERY:${query}$ CONTEXT:${context}$'''

    response = model.generate_content(formatted_response)
    print(response.text)



Hello Amar,

Thank you for reaching out! 🌟 Based on the information you provided, here are some government schemes that might be relevant to your situation:

1. Dr. B.R. Ambedkar Awas Navinikarn Yojna, Haryana:
   - **Eligibility:** Families living under the poverty line who own a house in need of repair.
   - **Benefit:** Grant for house repairs.

2. General Grant-in-Aid Scheme in the field of Women and Child Development:
   - **Eligibility:** Voluntary organizations working in the field of women and child development.
   - **Benefit:** Financial assistance for projects and activities.

3. Entrepreneurial Schemes of National Backward Classes Finance and Development:
   - **Eligibility:** Individuals belonging to backward classes who wish to start or expand businesses.
   - **Benefit:** Loans at concessional rates of interest, skill development training.

4. National Safai Karamcharis Finance & Development Corporation:
   - **Eligibility:** Safai Karamcharis, manual scavengers, and the